<a href="https://colab.research.google.com/github/gitcloned/colab-notebooks/blob/master/RVL%20CDIP%20Trained%20Models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
import os
import numpy as np

In [2]:
from keras import backend as K
# from keras.utils.layer_utils import convert_all_kernels_in_model
from keras.applications import VGG16 

Using TensorFlow backend.


In [3]:
K.set_image_data_format('channels_first')
th_dim_model = VGG16(include_top=True, weights=None, input_shape=(3,224,224), pooling=None, classes=16) # Create your theano model here with TH dim ordering

K.set_image_data_format('channels_last')
tf_dim_model = VGG16(include_top=True, weights=None, input_shape=(224,224,3), pooling=None, classes=16) # Create your tensorflow model with TF dimordering here


W0827 19:08:55.609333 140159198549888 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0827 19:08:55.671626 140159198549888 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0827 19:08:55.679924 140159198549888 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0827 19:08:55.701600 140159198549888 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.

W0827 19:08:55.702610 1401591985

In [0]:
model_weights = ['vgg16_weights_th_dim_ordering_th_kernels_Holistic_91.11.h5']

In [0]:
def shuffle_rows(original_w, nb_last_conv, nb_rows_dense):
    ''' Note :
    This algorithm to shuffle dense layer rows was provided by Kent Sommers (@kentsommer)
    in a gist : https://gist.github.com/kentsommer/e872f65926f1a607b94c2b464a63d0d3
    '''
    converted_w = np.zeros(original_w.shape)
    count = 0
    for index in range(original_w.shape[0]):
        if (index % nb_last_conv) == 0 and index != 0:
            count += 1
        new_index = ((index % nb_last_conv) * nb_rows_dense) + count
        #print("index from " + str(index) + " -> " + str(new_index))
        converted_w[index] = original_w[new_index]

    return converted_w


first_dense = True
nb_last_conv = 0

In [0]:
root_folder = '/content/gdrive/My Drive/Research/OCR/C0002/'
dirpath = root_folder + "tf-kernels-channels-last-dim-ordering/"
if not os.path.exists(dirpath):
    os.makedirs(dirpath)

In [14]:
K.set_image_dim_ordering('th')
for weight_fn in model_weights:
    th_dim_model.load_weights(root_folder + weight_fn) # th-kernels-th-dim
    #convert_all_kernels_in_model(th_dim_model) # tf-kernels-th-dim // ALREADY DONE BY KERAS!

    count_dense = 0
    for layer in th_dim_model.layers:
        if layer.__class__.__name__ == "Dense":
            count_dense += 1

    if count_dense == 1:
        first_dense = False # If there is only 1 dense, no need to perform row shuffle in Dense layer

    print("Nb layers : ", len(th_dim_model.layers))

    for index, th_layer in enumerate(th_dim_model.layers):
        if th_layer.__class__.__name__ in ['Conv1D',
                                           'Conv2D',
                                           'Conv3D',
                                           'AtrousConvolution1D'
                                           'AtrousConvolution2D',
                                           'Conv2DTranspose',
                                           'SeparableConv2D',
                                           'DepthwiseConv2D',
                                           ]:
            weights = th_layer.get_weights() # tf-kernels-th-dim
            #weights[0] = weights[0].transpose((2, 3, 1, 0)) // ALREADY DONE BY KERAS!
            tf_dim_model.layers[index].set_weights(weights) # tf-kernels-tf-dim

            nb_last_conv = th_layer.filters # preserve last number of convolutions to use with dense layers // UPDATED from nb_filters
            print("Converted layer %d : %s" % (index + 1, th_layer.name))
        else:
            if th_layer.__class__.__name__ == "Dense" and first_dense:
                weights = th_layer.get_weights()
                nb_rows_dense_layer = weights[0].shape[0] // nb_last_conv

                print("Magic Number 1 : ", nb_last_conv)
                print("Magic nunber 2 : ", nb_rows_dense_layer)

                weights[0] = shuffle_rows(weights[0], nb_last_conv, nb_rows_dense_layer)
                tf_dim_model.layers[index].set_weights(weights)

                first_dense = False
                print("Shuffled Dense Weights layer and saved %d : %s" % (index + 1, th_layer.name))
            else:
                tf_dim_model.layers[index].set_weights(th_layer.get_weights())
                print("Saved layer %d : %s" % (index + 1, th_layer.name))

  
    tf_dim_model.save_weights("/content/gdrive/My Drive/Research/OCR/C0002/tf-kernels-channels-last-dim-ordering/%s" % weight_fn, overwrite=True)
    print("Done tf-kernels-channels-last-dim-ordering %s" % weight_fn)

/usr/local/lib/python3.6/dist-packages/keras/utils/conv_utils.py:82: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  return np.copy(kernel[slices])


Nb layers :  23
Saved layer 1 : input_1
Converted layer 2 : block1_conv1
Converted layer 3 : block1_conv2
Saved layer 4 : block1_pool
Converted layer 5 : block2_conv1
Converted layer 6 : block2_conv2
Saved layer 7 : block2_pool
Converted layer 8 : block3_conv1
Converted layer 9 : block3_conv2
Converted layer 10 : block3_conv3
Saved layer 11 : block3_pool
Converted layer 12 : block4_conv1
Converted layer 13 : block4_conv2
Converted layer 14 : block4_conv3
Saved layer 15 : block4_pool
Converted layer 16 : block5_conv1
Converted layer 17 : block5_conv2
Converted layer 18 : block5_conv3
Saved layer 19 : block5_pool
Saved layer 20 : flatten
Saved layer 21 : fc1
Saved layer 22 : fc2
Saved layer 23 : predictions
Done tf-kernels-channels-last-dim-ordering vgg16_weights_th_dim_ordering_th_kernels_Holistic_91.11.h5
